
A DVD rental company needs to figure out how many days a customer will rent a DVD for based on some features. They want to predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

# Importing modules + quick dataset look

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [21]:
df=pd.read_csv('rental_info.csv')

In [3]:
df.isna().any()

rental_date         False
return_date         False
amount              False
release_year        False
rental_rate         False
length              False
replacement_cost    False
special_features    False
NC-17               False
PG                  False
PG-13               False
R                   False
amount_2            False
length_2            False
rental_rate_2       False
dtype: bool

In [4]:
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [5]:
df.sample(5)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
3263,2005-08-21 14:15:38+00:00,2005-08-28 09:50:38+00:00,2.99,2008.0,0.99,53.0,25.99,"{""Deleted Scenes"",""Behind the Scenes""}",0,0,0,0,8.9401,2809.0,0.9801
2986,2005-06-21 00:44:40+00:00,2005-06-23 23:10:40+00:00,2.99,2007.0,2.99,89.0,12.99,"{Trailers,""Behind the Scenes""}",0,0,0,0,8.9401,7921.0,8.9401
3783,2005-07-31 21:22:39+00:00,2005-08-01 22:55:39+00:00,0.99,2004.0,0.99,117.0,29.99,"{Trailers,""Deleted Scenes""}",0,1,0,0,0.9801,13689.0,0.9801
2698,2005-07-07 17:51:54+00:00,2005-07-10 13:43:54+00:00,0.99,2009.0,0.99,87.0,21.99,"{Commentaries,""Behind the Scenes""}",0,0,1,0,0.9801,7569.0,0.9801
4219,2005-07-30 15:24:14+00:00,2005-08-08 20:24:14+00:00,2.99,2005.0,0.99,52.0,11.99,"{Commentaries,""Deleted Scenes""}",1,0,0,0,8.9401,2704.0,0.9801


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


# Data cleaning

### Taking care of dates

In [22]:
#first, converting data type
df['rental_date']=pd.to_datetime(df['rental_date'])
df['return_date']=pd.to_datetime(df['return_date'])

'''I don't want to keep the hours and minutes in those 2 variables, 
but since it can be valuable info, i'm creating a new variable with 
the amount of time the user rented the movie.'''
df['days_rented']=df['return_date']-df['rental_date']
df['days_rented']=df['days_rented'].dt.total_seconds() / 86400

#now, leaving out the hours, minutes, etc
df['rental_date']=df['rental_date'].dt.strftime('%Y-%m-%d')
df['return_date']=df['return_date'].dt.strftime('%Y-%m-%d')

In [23]:
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,days_rented
0,2005-05-25,2005-05-28,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3.865278
1,2005-06-15,2005-06-18,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2.836806
2,2005-07-10,2005-07-17,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7.238889
3,2005-07-31,2005-08-02,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2.100000
4,2005-08-19,2005-08-23,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4.045139


### Changing datatypes

In [24]:
df['release_year']=df['release_year'].astype('int')

### Data visualization: Inspecting variables one by one

**Amount**

In [25]:
df.amount.describe()

count    15861.000000
mean         4.217161
std          2.360383
min          0.990000
25%          2.990000
50%          3.990000
75%          4.990000
max         11.990000
Name: amount, dtype: float64

In [8]:
px.histogram(df, x='amount', nbins=20)

**Release year**

In [11]:
df.release_year.value_counts(normalize=True)
#it's balanced (year-wise)

2004    0.166761
2006    0.157178
2007    0.153395
2010    0.139903
2009    0.138516
2005    0.132715
2008    0.111531
Name: release_year, dtype: float64

In [12]:
df.rental_rate.describe()

count    15861.000000
mean         2.944101
std          1.649766
min          0.990000
25%          0.990000
50%          2.990000
75%          4.990000
max          4.990000
Name: rental_rate, dtype: float64

In [13]:
df.length.describe()

count    15861.000000
mean       114.994578
std         40.114715
min         46.000000
25%         81.000000
50%        114.000000
75%        148.000000
max        185.000000
Name: length, dtype: float64

In [15]:
df.replacement_cost.describe()

count    15861.000000
mean        20.224727
std          6.083784
min          9.990000
25%         14.990000
50%         20.990000
75%         25.990000
max         29.990000
Name: replacement_cost, dtype: float64

In [9]:
px.histogram(df,x='replacement_cost',nbins=100)

In [10]:
df.special_features.value_counts()

{Trailers,Commentaries,"Behind the Scenes"}                     1308
{Trailers}                                                      1139
{Trailers,Commentaries}                                         1129
{Trailers,"Behind the Scenes"}                                  1122
{"Behind the Scenes"}                                           1108
{Commentaries,"Deleted Scenes","Behind the Scenes"}             1101
{Commentaries}                                                  1089
{Commentaries,"Behind the Scenes"}                              1078
{Trailers,"Deleted Scenes"}                                     1047
{"Deleted Scenes","Behind the Scenes"}                          1035
{"Deleted Scenes"}                                              1023
{Commentaries,"Deleted Scenes"}                                 1011
{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}     983
{Trailers,Commentaries,"Deleted Scenes"}                         916
{Trailers,"Deleted Scenes","Behind

In [26]:
mlb= MultiLabelBinarizer()

In [27]:
# first, preparing the variable values by taking care of spaces,{}, making everything lowercase...
df['special_features']=[c.lower().replace('"','').strip('{}').replace(' ','_') for c in df.special_features]

In [28]:
# second, converting the variable values into lists, so i can apply the mlb
df['special_features']=df.special_features.str.split(',')

In [29]:
# third, creating a df that has as many columns as different features are in 'special_features'
set_features=pd.DataFrame(mlb.fit_transform(df['special_features']),columns=mlb.classes_)

In [30]:
# and finally, merging those to my original df
df=df.join(set_features,how='left')

In [31]:
df=df.drop(['special_features','rental_date','return_date'], axis=1)

# Train-test split

In [32]:
x=df.drop('days_rented',axis=1)
y=df['days_rented']
x_train, y_train, x_test, y_test = train_test_split(x,y,test_size=0.1,random_state=42)

# Feature selection

In [ ]:
lasso= Lasso(alpha=0.1)
lasso.fit(x_train,y_train)
lasso_pred=lasso.predict(x_test, normalize=True)
lasso.score(x_test,y_test)

ValueError: Found input variables with inconsistent numbers of samples: [14274, 1587]